In [ ]:
import scvelo as scv
import numpy as np
import pandas as pd

import scanpy as sc
import matplotlib.pyplot as plt 

In [ ]:
adata = scv.datasets.pancreas()


In [ ]:
sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)


# Builtin HVG

In [ ]:
# adata.var['highly_variable_genes']

In [ ]:
# actual_mask = adata.var['highly_variable_genes'] == "True"

In [ ]:
# filtered = adata[:,actual_mask]

In [ ]:
# WHY IS VARIABILITY A MIXTURE OF BOOL STRINGS AND NANS? JFC
# filtered

# Foreign HVG

In [ ]:
hvg = np.loadtxt("./ZPSGenes.tsv",dtype=str)
hvg.shape

In [ ]:
filtered = adata[:,hvg]

### Let's just double check real quick if there was even minimal sanity when doing the hv selection

In [ ]:
sc.pp.highly_variable_genes(adata,n_top_genes=4004)
adata.var['highly_variable']

In [ ]:
sc_calls = adata.var_names[adata.var['highly_variable']]
foreign_calls = adata.var_names[actual_mask]

In [ ]:
np.sum(actual_mask.astype(dtype=int))

In [ ]:
len(set(sc_calls).intersection(set(foreign_calls)))

# Vignette Analysis

In [ ]:
sc.pp.neighbors(filtered)
sc.tl.umap(filtered)
sc.pl.umap(filtered)

In [ ]:
# filtered.obsm['X_umap'] = u_t1

In [ ]:
scv.pp.moments(filtered,n_neighbors=None,n_pcs=None)

In [ ]:
scv.tl.velocity(filtered)
scv.tl.velocity_graph(filtered)


In [ ]:
# scv.pl.velocity_embedding(filtered,basis='umap',figsize=((30,30)))

In [ ]:
# scv.pl.velocity_embedding_stream(filtered, density=5, basis='umap')

In [ ]:
# sc.pl.umap(filtered,color="S_score")
# sc.pl.umap(filtered,color="G2M_score")

In [ ]:
# scv.pl.proportions(filtered)

# Velocity Graph 

In [ ]:
u_t = filtered.obsm['X_umap']

In [ ]:
# We want this rather than the velocity graph in itself
tg = np.array(scv.utils.get_transition_matrix(filtered).todense())

In [ ]:
def mean_velo(tg,data):
    wsum = np.matmul(tg,data) 
    norm_const = np.sum(tg,axis=1)
    wmean = (wsum.T / norm_const).T
    velo = wmean - data
    return velo
    # length = np.sqrt(np.sum(np.power(velo,2),axis=1))
    # unitized = (velo.T / length).T
    # return unitized

In [ ]:
u_t_v = mean_velo(tg,u_t)

In [ ]:
def delta_graph(origin,delta,highlight=[0,1],arrow_frequency=30,figsize=(10,8)):

    plt.figure(figsize=figsize)
    plt.scatter(*origin.T,color='red',s=1)

    for i,((x,y),(dx,dy)) in enumerate(zip(origin,delta)):
        if i%arrow_frequency == 0:
            plt.text(*origin[i],s=f"{i}")
            plt.arrow(x,y,dx,dy,color='green',head_width=.1,linewidth=.2)

    plt.show()

In [ ]:
delta_graph(u_t,u_t_v/5,arrow_frequency=3,highlight=[3660,],figsize=(30,30))

In [ ]:
# Hallelujah, using the get_transition_matrix, we're not looking too bad. Can probably use this as an input to NODE using an arbitrary 
# embedding, and generate paired data with it without throwing up in my mouth too much 


In [ ]:
t0 = u_t + 5
t1 = u_t + 5 + (u_t_v/5)
np.savetxt("sc_velo_t0_emb.tsv",t0)
np.savetxt("sc_velo_t1_pred_emb.tsv",t1)

In [ ]:
# print(t0[3609])
# print(t1[3609])

# Foreign UMAP

In [ ]:
import matplotlib.pyplot as plt 

In [ ]:
from umap import UMAP
from sklearn.decomposition import PCA

In [ ]:
pca_model = PCA(n_components=50)
pca = pca_model.fit_transform(np.array(filtered.X.todense()))

umap_model = UMAP(n_neighbors=15,min_dist=0.5, spread=1.0, n_components=2, negative_sample_rate=5, random_state=0,metric='cosine')
u_t = umap_model.fit_transform(pca)

In [ ]:
plt.figure()
plt.scatter(*u_t.T,s=1)
# for i in range(0,3600,100):
#     plt.text(*u_t[i],f"{i}",)
plt.show()

In [ ]:
plt.figure()
plt.scatter(*filtered.obsm['X_umap'].T,s=1)
# for i in range(1,3601,100):
#     plt.text(*filtered.obsm['X_umap'][i],f"{i}")
plt.show()

In [ ]:
trajectories_1 = np.loadtxt("./trajectories_t1.tsv",dtype=float).T
trajectories_2 = np.loadtxt("./trajectories_t2.tsv",dtype=float).T
trajectories_3 = np.loadtxt("./trajectories_t3.tsv",dtype=float).T


In [ ]:
pca_transformed_trajectories_1 = pca_model.transform(trajectories_1)
pca_transformed_trajectories_2 = pca_model.transform(trajectories_2)
pca_transformed_trajectories_3 = pca_model.transform(trajectories_3)


embedded_1 = umap_model.transform(pca_transformed_trajectories_1)
embedded_2 = umap_model.transform(pca_transformed_trajectories_2)
embedded_3 = umap_model.transform(pca_transformed_trajectories_3)


In [ ]:

plt.figure(figsize=(10,8))
plt.scatter(*u_t.T,s=1)
for i in range(0,embedded_1.shape[0],30): 
    plt.arrow(*arrow_deltas(u_t[i],embedded_1[i]),color='green',head_width=.1,linewidth=.2)
    plt.arrow(*arrow_deltas(embedded_1[i],embedded_2[i]),color='red',head_width=.1,linewidth=.2)
    # plt.plot([u_t[i,0],embedded[i,0]],[u_t[i,1],embedded[i,1]],color='red')
plt.show()

In [ ]:
# trajectories = np.loadtxt("./trajectories_1700.tsv",dtype=float).T
# trajectories.shape

In [ ]:
simple_sequence(t1,trajectories,highlight=[],arrow_frequency=10)

In [ ]:
pca_transformed_trajectories = pca_model.transform(trajectories)
pca_transformed_trajectories.shape

In [ ]:
embedded = umap_model.transform(pca_transformed_trajectories)
embedded.shape

In [ ]:
plt.figure(figsize=(10,8))
plt.scatter(*u_t.T,s=1)
for i in range(0,embedded.shape[0],10):
    x,y = u_t[i]
    dx,dy = (u_t[i] - embedded[i]) / 2
    plt.arrow(x,y,dx,dy,color='red',head_width=.1,linewidth=.2)
    # plt.plot([u_t[i,0],embedded[i,0]],[u_t[i,1],embedded[i,1]],color='red')
plt.show()

In [ ]:
# plt.figure()
# plt.scatter(*u_t.T,s=1)
# plt.plot(*embedded[:50].T,color='red')
# plt.show()

In [ ]:
trajectories = trajectories.reshape((1352,2,50))

In [ ]:
trajectories

# Predicted Foreign Umap

In [ ]:
t_1 = np.loadtxt("./trajectories_t1.tsv",dtype=float).T
t_2 = np.loadtxt("./trajectories_t2.tsv",dtype=float).T
t_3 = np.loadtxt("./trajectories_t3.tsv",dtype=float).T
t_4 = np.loadtxt("./trajectories_t4.tsv",dtype=float).T
t_5 = np.loadtxt("./trajectories_t5.tsv",dtype=float).T
t_6 = np.loadtxt("./trajectories_t6.tsv",dtype=float).T


In [ ]:
def arrow_deltas(t1,t2):
    x,y = t1
    dx,dy = t2 - t1
    return (x,y,dx,dy)

def trajectory_series(trajectories,frequency=5):
    plt.figure(figsize=(30,30))
    plt.scatter(*trajectories[0].T,s=3)
    for t1,t2 in zip(trajectories[:-1],trajectories[1:]):
        for i in range(0,t1.shape[0],frequency): 
            plt.arrow(*arrow_deltas(t1[i],t2[i]),color='green',head_width=.05,linewidth=.2)
    plt.show()    

trajectory_series([u_t + 5,t_1,t_2,t_3,t_4,t_5,t_6])
    
# plt.figure(figsize=(30,30))
# plt.scatter(*u_t.T + 5,s=1)
# for i in range(0,trajectories_1.shape[0],30): 
#     plt.arrow(*arrow_deltas(u_t[i] + 5,trajectories_1[i]),color='red',head_width=.1,linewidth=.2)
#     plt.arrow(*arrow_deltas(trajectories_1[i],trajectories_2[i]),color='green',head_width=.1,linewidth=.2)
#     plt.arrow(*arrow_deltas(trajectories_2[i],trajectories_3[i]),color='blue',head_width=.1,linewidth=.2)
# plt.show()

In [ ]:
np.max(t_5,axis=0)

In [ ]:
# torubleshooting offsets

# some trajectory starts at ~ 4,10.5
# First let's index it, it should be easy to find 

t1


# Joint Embedding?

In [ ]:
t1 = np.array(filtered.X.copy().todense())

# # Is this in log space?
v = np.array(filtered.layers['velocity'])
v = np.log1p(np.abs(v)) * np.sign(v)

t2 = t1+(v/5)

# clip negatives

t2[t2 < 0] = 0

In [ ]:
from umap import UMAP
from sklearn.decomposition import PCA

def arrow_deltas(time1,time2,damping=1):
    x,y = time1
    dx,dy = time2 - time1
    return (x,y,dx*damping,dy*damping)
    
def simple_sequence(time1,time2,highlight=[0,1],arrow_frequency=30):
    pca_model = PCA(n_components=50)
    pca_t1 = pca_model.fit_transform(time1)
    pca_t2 = pca_model.transform(time2)
    
    umap_model = UMAP(n_neighbors=15,min_dist=0.5, spread=1.0, n_components=2, negative_sample_rate=5, random_state=0,metric='cosine')
    u_t1 = umap_model.fit_transform(pca_t1)
    u_t2 = umap_model.transform(pca_t2)

    plt.figure(figsize=(10,8))
    plt.scatter(*u_t1.T,color='red',s=1)
    # plt.show()
    # plt.figure(figsize=(10,8))
    plt.scatter(*u_t2.T,color='blue',s=1)
    plt.show()

    # plt.figure(figsize=(10,8))
    # plt.scatter(*u_t1.T,s=1)
    # for i in highlight:
    #     plt.text(*u_t1[i],s=f"{i}")
    #     plt.text(*u_t2[i],s=f"{i}'")
    #     plt.arrow(*arrow_deltas(u_t1[i],u_t2[i]),color='red',head_width=.1,linewidth=.2)
    # for i in range(0,u_t1.shape[0],arrow_frequency): 
    #     plt.arrow(*arrow_deltas(u_t1[i],u_t2[i]),color='green',head_width=.1,linewidth=.2)
    # plt.show()

    return u_t1
    
    # plt.figure(figsize=(10,8))
    # plt.scatter(*pca_t1[:,:2].T,s=1)
    # for i in highlight:
    #     plt.text(*pca_t1[i,:2],s=f"{i}")
    # for i in range(0,pca_t1[:,:2].shape[0],arrow_frequency): 
    #     plt.arrow(*arrow_deltas(pca_t1[i,:2],pca_t2[i,:2],damping=1),color='green',head_width=.1,linewidth=.2)
    # plt.show()